$$
\renewcommand{\d}[1]{\mathrm{d} #1 \,}
\newcommand{\dern}[3]{\frac{\mathrm{d}^{#3} #1}{\mathrm{d} #2^{#3}}}
\newcommand{\bra}[1]{\left\langle{#1}\right|}
\newcommand{\ket}[1]{\left|{#1}\right\rangle}
\newcommand{\braket}[1]{\left\langle{#1}\right\rangle}
$$

# Variationeel principe: oneindig diepe potentiaalput

## Opgave

Beschouw een deeltje in een geperturbeerde oneindig diepe potentiaalput met breedte $L$, waarvan de potentiële energie gegeven wordt door
	\begin{equation}
		V(x) = \begin{cases}
				a x^p, & - \frac{L}{2} < x < \frac{L}{2} \\
				\infty, & |x| \geq \frac{L}{2}
				\end{cases},
	\end{equation}
	
met $p$ een natuurlijk getal en $a L^p \ll 2^p$ zodat de afwijking van de oneindig diepe potentiaalput klein is. Aangezien de potentiaal oneindig groot wordt op de randen van de potentiaalput, dient de golffunctie daar te verdwijnen. Als variationele golffunctie kiezen we
	\begin{equation}
		\psi(x) = \sum_{n=0}^{N-1} C_n \,x^n \left( x^2 - \frac{L^2}{4} \right).
	\end{equation}
	
1.	Vergelijk voor de oneindig diepe potentiaalput ($a = 0$) de variationele energie-eigenwaarden en eigenfuncties met de exacte analytische oplossing (*vide infra*) voor verschillende waarden van $N$. Verklaar ook het convergentiegedrag als functie van $N$.

2.	Vergelijk voor $p = 1$ de variationele energie-eigenwaarden en eigenfuncties met de resultaten uit Oefening 4 (geladen deeltje in 1D condensator) voor verschillende waarden van $N$.


## Oplossing

Eerst implementeren we een functie `calc_matrices` die de matrices $H$ en $S$ genereert, gebruik makend van de volgende matrixelementen:
\begin{align}
    S_{mn} & = \braket{\chi_m | \chi_n} = \int_{-\frac{L}{2}}^{\frac{L}{2}} \d{x} \chi_m(x) \, \chi_n(x) \nonumber \\
    & = \left( \frac{L}{2} \right)^{m+n+5} \frac{16}{(m+n+5) (m+n+3) (m+n+1)} \; \frac{1 + (-1)^{m+n}}{2} \\[0.25cm]
    T_{mn} & = \bra{\chi_m} \hat{T} \ket{\chi_n} = - \frac{1}{2} \int_{-\frac{L}{2}}^{\frac{L}{2}} \d{x} \chi_m(x) \; \dern{}{x}{2} \,\chi_n(x) \nonumber \\
    & = - 4 \left( \frac{L}{2} \right)^{m+n+3} \frac{1 - m - n - 2mn}{(m+n+3) (m+n+1) (m+n-1)} \; \frac{1 + (-1)^{m+n}}{2} \\[0.25cm]
    V_{mn} & = \bra{\chi_m} \hat{V} \ket{\chi_n} = a \int_{-\frac{L}{2}}^{\frac{L}{2}} \d{x} \chi_m(x) \, x^p \,\chi_n(x) \nonumber \\
    & = \left( \frac{L}{2} \right)^{m+n+p+5} \frac{16a}{(m+n+p+5) (m+n+p+3) (m+n+p+1)} \; \frac{1 + (-1)^{m+n+p}}{2}
\end{align}

In [ ]:
import numpy as np
from scipy.linalg import eigh
import matplotlib.pyplot as pl
%matplotlib inline

# Conversiefactoren atomaire eenheden
kjmol = 0.0003808799176039228
electronvolt = 0.036749325919680595

def calc_matrices(N=5, L=1., a=0., p=0):
    
    n = np.arange(0, N)
    nplusm = n + np.expand_dims(n, axis=1)

    S = (0.5*L)**(nplusm + 5) * 8/((nplusm + 5)*(nplusm + 3)*(nplusm + 1))
    S *= (-1)**nplusm + 1
    T = -2 * (0.5*L)**(nplusm + 3) * (1 - nplusm - 2*n*np.expand_dims(n, axis=1))/((nplusm + 3)*(nplusm + 1))
    T = np.divide(T, nplusm - 1, out=np.zeros(T.shape), where=(nplusm - 1)!=0)
    T *= (-1)**nplusm + 1

    V = (0.5*L)**(nplusm + p + 5) * 8*a/((nplusm + p + 5)*(nplusm + p + 3)*(nplusm + p + 1))
    V *= (-1)**(nplusm + p) + 1

    return T + V, S

Vervolgens vergelijken we de variationele oplossing van de oneindig diepe potentiaalput met de exacte analytische oplossing:
\begin{align}
     E_n^{(0)} & = \frac{\hbar^2}{2 m} \, \frac{n^2 \pi^2}{L^2} \\
    \psi_n^{(0)}(x) & = \sqrt{\frac{2}{L}} \sin \left(\frac{n \pi x}{L}\right) && \qquad \text{$n$ even} \label{oef3_psi} \\
    \psi_n^{(0)}(x) & = \sqrt{\frac{2}{L}} \cos \left(\frac{n \pi x}{L}\right) && \qquad \text{$n$ oneven} 
\end{align}

De functie `eigh` kan gebruikt worden om het veralgemeende eigenwaardeprobleem op te lossen.

In [ ]:
# Variationele oplossing
Nmax = 20
L = 1.

energy = np.zeros((Nmax, Nmax))
for N in np.arange(1, Nmax+1):
    H, S = calc_matrices(N, L=L, a=0.)
    E, C = eigh(H, S)
    energy[N-1, :N] = E
    
# Exacte oplossing
kn = np.arange(1, Nmax+1)*np.pi/L
E_exact = kn**2/2

x = np.arange(-L/2, L/2 + 0.01, 0.01)

psi = np.zeros((N, len(x)))
psi[0::2, :] = np.sqrt(2/L) * np.cos(np.expand_dims(kn[0::2], axis=1)*x)
psi[1::2, :] = np.sqrt(2/L) * np.sin(np.expand_dims(kn[1::2], axis=1)*x)

In [ ]:
# Plot convergentie energieniveaus
for n in np.arange(min(8, Nmax+1)):
    pl.plot(np.arange(n, Nmax) + 1, (energy[n:,n]-E_exact[n])/kjmol, 'o--', label='$E_{{{}}}$'.format(n+1))

pl.xlabel('$N$')
pl.ylabel('$\\varepsilon_n - E_{n,\ exact}$ (kJ/mol)')
pl.xticks(np.arange(0, Nmax+1))
pl.ylim([-0.05, 5.0])
pl.legend()
pl.title('Oneindig diepe potentiaalput')
pl.show()

In [ ]:
# Plot golffuncties
for i in range(N):

    psi_var = np.dot(C[:, i], (x**2 - 0.25*L**2) * x**np.expand_dims(np.arange(0, N), axis=1))
    sign_corr = np.sign(psi[i, int(psi.shape[1]/2.5)]/psi_var[int(psi.shape[1]/2.5)])
    
    pl.plot(x, psi[i, :], label='Variationeel')
    pl.plot(x, sign_corr*psi_var, '--', label='Analytisch')
    pl.title('$n =$ {}, $\Delta E =$ {:.3f} kJ/mol'.format(i+1, (E-E_exact)[i]/kjmol))
    pl.legend()
    pl.show()

Tot slot vergelijken we ook de resultaten van het computationele variationeel principe voor de golffunctie
\begin{equation}
    \psi(x) = \sum_{n=0}^{N-1} C_n \,x^n \left( x^2 - \frac{L^2}{4} \right),
\end{equation}

met de variationele resultaten uit Oefening 4, waarbij gebruik werd gemaakt van de volgende golffunctie:
\begin{equation}
    \psi_{\text{oef. 4}}(x) = \psi_1^{(0)}(x) \, \cos \theta + \psi_2^{(0)}(x) \, \sin \theta.
\end{equation}

In [ ]:
# Oplossing oefening 4
e = 0.5*(np.pi/L)**2
v = 16*L/(9*np.pi**2)
E_oef4 = e*(2.5 - 1.5*np.sqrt(1 + 4*v**2/(9*e**2)))

# Variationele oplossing
Nmax = 10
E0 = np.zeros(Nmax)
for N in np.arange(1, Nmax+1):
    H, S = calc_matrices(N, L=L, a=1., p=1)
    E, C = eigh(H, S)
    E0[N-1] = E[0]

pl.plot(np.arange(Nmax) + 1, (E0-E_oef4)/kjmol, 'o--')
pl.xlabel('$N$')
pl.ylabel('$\\varepsilon_0 - E_{0, oef.\ 4}}$ (kJ/mol)')
pl.ylim([-0.05, 1.05])
pl.show()